In [ ]:
import os
import pickle
import numpy as np
import torch
import torch.nn.functional as F
from collections import OrderedDict
from onmt.utils.misc import sequence_mask
from model_autopst import Generator_2 as Predictor
from hparams_autopst import hparams

device = 'cuda:0'

P = Predictor(hparams).eval().to(device)

checkpoint = torch.load('./assets/580000-P.ckpt', map_location=lambda storage, loc: storage)  
P.load_state_dict(checkpoint['model'], strict=True)
print('Loaded predictor .....................................................')

dict_test = pickle.load(open('./assets/test_vctk.meta', 'rb'))

In [ ]:
spect_vc = OrderedDict()

uttrs = [('p231', 'p270', '001'),
         ('p270', 'p231', '001'),
         ('p231', 'p245', '003001'),
         ('p245', 'p231', '003001'),
         ('p239', 'p270', '024002'),
         ('p270', 'p239', '024002')]


for uttr in uttrs:
        
    cep_real, spk_emb = dict_test[uttr[0]][uttr[2]]
    cep_real_A = torch.from_numpy(cep_real).unsqueeze(0).to(device)
    len_real_A = torch.tensor(cep_real_A.size(1)).unsqueeze(0).to(device)
    real_mask_A = sequence_mask(len_real_A, cep_real_A.size(1)).float()
    
    _, spk_emb = dict_test[uttr[1]][uttr[2]]
    spk_emb_B = torch.from_numpy(spk_emb).unsqueeze(0).to(device)
    
    with torch.no_grad():
        spect_output, len_spect = P.infer_onmt(cep_real_A.transpose(2,1)[:,:14,:],
                                               real_mask_A,
                                               len_real_A,
                                               spk_emb_B)
    
    uttr_tgt = spect_output[:len_spect[0],0,:].cpu().numpy()
        
    spect_vc[f'{uttr[0]}_{uttr[1]}_{uttr[2]}'] = uttr_tgt

In [ ]:
# spectrogram to waveform
# Feel free to use other vocoders
# This cell requires some preparation to work, please see the corresponding part in AutoVC
import torch
import librosa
import pickle
import os
from synthesis import build_model
from synthesis import wavegen

model = build_model().to(device)
checkpoint = torch.load("./assets/checkpoint_step001000000_ema.pth")
model.load_state_dict(checkpoint["state_dict"])

for name, sp in spect_vc.items():
    print(name)
    waveform = wavegen(model, c=sp)   
    librosa.output.write_wav('./assets/'+name+'.wav', waveform, sr=16000)